In [ ]:
import sys
import os
import findspark
findspark.init()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
import spark
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [ ]:
user = os.getenv("user", default=None)
password = os.getenv("password", default=None)

In [ ]:
spark = SparkSession.builder.appName('capstone_project').getOrCreate()

In [ ]:
cdw_sapp_customer = os.path.join("files", "cdw_sapp_custmer.json")
df_customer = spark.read.json(cdw_sapp_customer)

In [ ]:
type(df_customer)
df_customer.show()

In [ ]:
df_customer.printSchema()

In [ ]:
df_customer.createOrReplaceTempView('customer_data')

In [ ]:
customer_df=spark.sql("SELECT CAST(SSN AS INT) SSN, \
CONCAT(UCASE(SUBSTRING(FIRST_NAME,1,1)), LCASE(SUBSTRING(FIRST_NAME,2))) FIRST_NAME, \
LOWER(MIDDLE_NAME) MIDDLE_NAME, \
CONCAT(UCASE(SUBSTRING(last_name,1,1)), LCASE(SUBSTRING(last_name,2))) LAST_NAME, \
Credit_card_no, \
CONCAT(APT_NO,' ,',STREET_NAME) FULL_STREET_ADDRESS, \
CUST_CITY, CUST_STATE, CUST_COUNTRY, CUST_ZIP, CONCAT(SUBSTRING(cust_phone, 1,3), '-',SUBSTRING(cust_phone,4)) CUST_PHONE, \
CUST_EMAIL, LAST_UPDATED FROM customer_data")

In [ ]:
customer_df.show() 

In [ ]:
customer_df.printSchema()

In [ ]:
#df_customer.createOrReplaceTempView('customer_df')
customer_df.columns

In [ ]:
customer_df.createOrReplaceTempView('customer_df')

In [ ]:
customer_df=spark.sql("SELECT SSN, FIRST_NAME, MIDDLE_NAME, LAST_NAME, Credit_Card_no, FULL_STREET_ADDRESS, CUST_CITY,CUST_STATE, \
CUST_COUNTRY, CAST(CUST_ZIP as INT) CUST_ZIP, CUST_PHONE, CUST_EMAIL, LAST_UPDATED FROM customer_df")

In [ ]:
customer_df.printSchema()

In [ ]:
customer_df.show()

In [ ]:
df_customer = customer_df.withColumn('LAST_UPDATED', F.to_timestamp('LAST_UPDATED'))

In [ ]:
df_customer.printSchema()

In [ ]:
df_customer.show()

In [ ]:
# 'DROP TABLE `creditcard_capstone`.`cdw_sapp_customer`; '

In [ ]:
df_customer.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.cdw_sapp_customer") \
.option("user", user) \
.option("password", password) \
.save()

In [ ]:
spark.stop()

In [ ]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_customer` 
# CHANGE COLUMN `FIRST_NAME` `FIRST_NAME` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `MIDDLE_NAME` `MIDDLE_NAME` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `LAST_NAME` `LAST_NAME` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `Credit_Card_no` `Credit_Card_no` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `FULL_STREET_ADDRESS` `FULL_STREET_ADDRESS` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `CUST_CITY` `CUST_CITY` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `CUST_STATE` `CUST_STATE` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `CUST_COUNTRY` `CUST_COUNTRY` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `CUST_PHONE` `CUST_PHONE` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `CUST_EMAIL` `CUST_EMAIL` VARCHAR(45) NULL DEFAULT NULL ;

In [ ]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_customer` 
# CHANGE COLUMN `SSN` `SSN` INT(11) NOT NULL ,
# ADD PRIMARY KEY (`SSN`);